In [1]:
# Implementation of back tracking algorithm
# Use crossword_puzzle.py for complete function and output generation

In [2]:
import numpy as np
import collections
from itertools import groupby

In [3]:
grid = []
with open('input.txt','r') as f:
    for i in range(10):
        grid.append(f.readline().strip())
    words = f.readline().strip().split(';')

In [4]:
grid

['+-++++++++',
 '+-++++++++',
 '+-++++++++',
 '+-----++++',
 '+-+++-++++',
 '+-+++-++++',
 '+++++-++++',
 '++------++',
 '+++++-++++',
 '+++++-++++']

In [5]:
words

['LONDON', 'DELHI', 'ICELAND', 'ANKARA']

In [6]:
[list(row) for row in grid]

[['+', '-', '+', '+', '+', '+', '+', '+', '+', '+'],
 ['+', '-', '+', '+', '+', '+', '+', '+', '+', '+'],
 ['+', '-', '+', '+', '+', '+', '+', '+', '+', '+'],
 ['+', '-', '-', '-', '-', '-', '+', '+', '+', '+'],
 ['+', '-', '+', '+', '+', '-', '+', '+', '+', '+'],
 ['+', '-', '+', '+', '+', '-', '+', '+', '+', '+'],
 ['+', '+', '+', '+', '+', '-', '+', '+', '+', '+'],
 ['+', '+', '-', '-', '-', '-', '-', '-', '+', '+'],
 ['+', '+', '+', '+', '+', '-', '+', '+', '+', '+'],
 ['+', '+', '+', '+', '+', '-', '+', '+', '+', '+']]

In [7]:
def extract_slots(grid, num_rows=10, num_cols=10):
    """
    Extract slots from grid so that words can be put in those slots
    return: list of lists of (i,j) such that positions in same slot
    are in the same list
    """
    grid = [list(row) for row in grid]
    res = []
    # Go through rows
    for r in range(num_rows):
        for k,g in groupby(enumerate(grid[r]),key=lambda x: x[1]):
            groups = list(g)
            if groups[0][1] == '-':
                res.append([(r,i) for i,e in groups])
    # Go through columns (or rows of transpose)
    for r in range(num_cols):
        for k,g in groupby(enumerate(np.array(grid).T[r]),key=lambda x: x[1]):
            groups = list(g)
            if groups[0][1] == '-':
                res.append([(i,r) for i,e in groups])
    
    # Output result as dictionary with keys as length of slot
    res_dict = collections.defaultdict(list)
    for slot in res:
        res_dict[len(slot)].append(slot)
    return dict(res_dict)

In [8]:
slots = extract_slots(grid)
slots

{1: [[(0, 1)],
  [(1, 1)],
  [(2, 1)],
  [(4, 1)],
  [(4, 5)],
  [(5, 1)],
  [(5, 5)],
  [(6, 5)],
  [(8, 5)],
  [(9, 5)],
  [(3, 2)],
  [(7, 2)],
  [(3, 3)],
  [(7, 3)],
  [(3, 4)],
  [(7, 4)],
  [(7, 6)],
  [(7, 7)]],
 5: [[(3, 1), (3, 2), (3, 3), (3, 4), (3, 5)]],
 6: [[(7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)],
  [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]],
 7: [[(3, 5), (4, 5), (5, 5), (6, 5), (7, 5), (8, 5), (9, 5)]]}

In [9]:
def is_valid(word, slot, assignment):
    """
    Check whether assignment is valid
    Args:
        word: word to be assigned
        slot: list of grid coordinate to assign letters to
        assignment: current assignment
    Return: boolean
    """
    if len(word) != len(slot):
        raise ValueError("Word and slot do not have same length")
    
    for i, letter in enumerate(word):
        if (slot[i] in assignment) and assignment[slot[i]] != letter:
            return False
    return True

def assign(word, slot, assignment):
    """
    Assign word letters to slot
    """
    for i, letter in enumerate(word):
        assignment[slot[i]] = letter
        
def unassign(word, slot, assignment):
    """
    Unassign word letter from already assigned slot
    """
    for i, letter in enumerate(word):
        del assignment[slot[i]]

def bt(words, i, slots, assignment, indent=""):
    # If all words are assigned, return True
    if i >= len(words):
        return True
    
    # If there are words left to be assigned, start assigning
    # to each slot, then check whether that assignment is valid
    w = words[i]
    for slot in slots[len(w)]:
        print(indent, w, slot, '::: valid =', is_valid(w, slot, assignment))
        if is_valid(w, slot, assignment):
            assign(w, slot, assignment)
            if bt(words, i+1, slots, assignment, indent+" |  ") == True:
                return True
            unassign(w, slot, assignment)
    return False

In [10]:
assignment = {}
bt(words, 0, slots, assignment)

 LONDON [(7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)] ::: valid = True
 |   DELHI [(3, 1), (3, 2), (3, 3), (3, 4), (3, 5)] ::: valid = True
 |   |   ICELAND [(3, 5), (4, 5), (5, 5), (6, 5), (7, 5), (8, 5), (9, 5)] ::: valid = False
 LONDON [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)] ::: valid = True
 |   DELHI [(3, 1), (3, 2), (3, 3), (3, 4), (3, 5)] ::: valid = True
 |   |   ICELAND [(3, 5), (4, 5), (5, 5), (6, 5), (7, 5), (8, 5), (9, 5)] ::: valid = True
 |   |   |   ANKARA [(7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)] ::: valid = True


True

In [11]:
assignment

{(0, 1): 'L',
 (1, 1): 'O',
 (2, 1): 'N',
 (3, 1): 'D',
 (3, 2): 'E',
 (3, 3): 'L',
 (3, 4): 'H',
 (3, 5): 'I',
 (4, 1): 'O',
 (4, 5): 'C',
 (5, 1): 'N',
 (5, 5): 'E',
 (6, 5): 'L',
 (7, 2): 'A',
 (7, 3): 'N',
 (7, 4): 'K',
 (7, 5): 'A',
 (7, 6): 'R',
 (7, 7): 'A',
 (8, 5): 'N',
 (9, 5): 'D'}

In [12]:
grid_res = [list(r) for r in grid]
for pos, letter in assignment.items():
    i, j = pos
    grid_res[i][j] = letter

In [13]:
print('\n'.join([''.join(r) for r in grid_res]))

+L++++++++
+O++++++++
+N++++++++
+DELHI++++
+O+++C++++
+N+++E++++
+++++L++++
++ANKARA++
+++++N++++
+++++D++++
